In [1]:
import os
import cooler
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import cm as cm
import seaborn as sns
import subprocess

In [3]:
#group_name = 'Oligo_NN'

In [4]:
ct = group_name
mode= 'raw'

In [5]:
comp = pd.read_hdf(f'{mode}_{ct}_celltyperawpca.hdf')
comp.columns = [_column.replace('.','_').replace('-','_') for _column in comp.columns]
comp.head()

,Oligo_NN_8wk,Oligo_NN_18mo,Oligo_NN_9mo
chr1-33,0.741294,2.191247,1.384147
chr1-34,1.014298,2.452876,0.075068
chr1-35,0.085219,0.387921,-0.358632
chr1-36,0.032600,-0.646450,-0.231420
chr1-39,-4.581002,-8.904725,-8.073639


In [6]:
leg = [f'{ct}.8wk',f'{ct}.9mo',f'{ct}.18mo']
leg = [_leg.replace('.','_').replace('-','_') for _leg in leg]
leg = pd.Index(leg)
leg

Index(['Oligo_NN_8wk', 'Oligo_NN_9mo', 'Oligo_NN_18mo'], dtype='object')

In [7]:
binall = pd.read_csv('binall.csv', index_col = 0)
binall

,chrom,start,end
chr1-33,chr1,3300000,3400000
chr1-34,chr1,3400000,3500000
chr1-35,chr1,3500000,3600000
chr1-36,chr1,3600000,3700000
chr1-39,chr1,3900000,4000000
...,...,...,...
chrX-1694,chrX,169400000,169500000
chrX-1695,chrX,169500000,169600000
chrX-1696,chrX,169600000,169700000
chrX-1697,chrX,169700000,169800000


In [8]:
res = 100000
chrom_size_path = '/ref/m3C/mm10.main.nochrM.nochrY.chrom.sizes'
chrom_sizes = cooler.read_chromsizes(chrom_size_path, all_names=True)

In [9]:
outdir = os.getcwd()
outdir

'/home/qzeng_salk_edu/project/240204-redo-compartment/template-ct-age'

In [10]:
for xx in leg:
    os.makedirs(f'{outdir}/{xx}_100Kb_pca/intra_pca/{xx}_100Kb_mat/', exist_ok=True)
    tmp = binall.copy()
    tmp['pc'] = comp[xx]
    for c in chrom_sizes.index:
        tmp.loc[tmp['chrom']==c].to_csv(f'{outdir}/{xx}_100Kb_pca/intra_pca/{xx}_100Kb_mat/{c}.pc.bedGraph', sep='\t', header=False, index=False)

In [11]:
tmp = pd.DataFrame(index=leg)
tmp['matrix_path'] = '.'
tmp['bed_path'] = '.'
tmp['sample'] = tmp.index + '_100Kb'
tmp['group'] = tmp.index
tmp.to_csv(f'{outdir}/input.txt', sep='\t', header=False, index=False)
tmp

,matrix_path,bed_path,sample,group
Oligo_NN_8wk,.,.,Oligo_NN_8wk_100Kb,Oligo_NN_8wk
Oligo_NN_9mo,.,.,Oligo_NN_9mo_100Kb,Oligo_NN_9mo
Oligo_NN_18mo,.,.,Oligo_NN_18mo_100Kb,Oligo_NN_18mo


In [12]:
rscript_path = '/home/qzeng_salk_edu/project/240204-redo-compartment/dchicf.r'

In [15]:
subprocess.run(f"Rscript {rscript_path} --file input.txt --pcatype analyze --dirovwt T --diffdir .", shell = True)

Running intra sample differential calls for  Oligo_NN.8wk Oligo_NN.9mo Oligo_NN.18mo  samples
Wrote  DifferentialResult/./Oligo_NN.8wk_data/intra_chr1_combined.pcOri.bedGraph and pcQnm.bedGraph files
Wrote  DifferentialResult/./Oligo_NN.8wk_data/intra_chr10_combined.pcOri.bedGraph and pcQnm.bedGraph files
Wrote  DifferentialResult/./Oligo_NN.8wk_data/intra_chr11_combined.pcOri.bedGraph and pcQnm.bedGraph files
Wrote  DifferentialResult/./Oligo_NN.8wk_data/intra_chr12_combined.pcOri.bedGraph and pcQnm.bedGraph files
Wrote  DifferentialResult/./Oligo_NN.8wk_data/intra_chr13_combined.pcOri.bedGraph and pcQnm.bedGraph files
Wrote  DifferentialResult/./Oligo_NN.8wk_data/intra_chr14_combined.pcOri.bedGraph and pcQnm.bedGraph files
Wrote  DifferentialResult/./Oligo_NN.8wk_data/intra_chr15_combined.pcOri.bedGraph and pcQnm.bedGraph files
Wrote  DifferentialResult/./Oligo_NN.8wk_data/intra_chr16_combined.pcOri.bedGraph and pcQnm.bedGraph files
Wrote  DifferentialResult/./Oligo_NN.8wk_data/intra

CompletedProcess(args='Rscript /home/qzeng_salk_edu/project/240204-redo-compartment/dchicf.r --file input.txt --pcatype analyze --dirovwt T --diffdir .', returncode=0)

In [2]:
# Rscript  /home/qzeng_salk_edu/project/230907-call_diff-compartment/dchicf.r --file input.txt --pcatype analyze --dirovwt T --diffdir .